In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
import patsy
from scipy.optimize import minimize
from scipy.linalg import block_diag

# -----------------------------------------------------------
# 1. Data Preprocessing
# -----------------------------------------------------------

meta = pd.read_csv("106_total.csv")
lasso_min_selected = pd.read_csv("lmin_total.csv")

In [2]:
# Type correction
meta['age'] = pd.to_numeric(meta['age'])
meta['sex'] = meta['sex'].astype('category')
meta['bmi'] = pd.to_numeric(meta['bmi'])
meta['days'] = pd.to_numeric(meta['days'])
meta['ID'] = meta['ID'].astype(str)

# Train/Test Split (8:2)
unique_ids = meta['ID'].unique()
np.random.seed(12)
train_ids = np.random.choice(unique_ids, size=int(len(unique_ids) * 0.8), replace=False)
test_ids = np.setdiff1d(unique_ids, train_ids)

train = meta[meta['ID'].isin(train_ids)].copy()
test = meta[meta['ID'].isin(test_ids)].copy()

In [3]:
# -----------------------------------------------------------
# 2. Feature Selection & Formula
# -----------------------------------------------------------

features_o = lasso_min_selected['Oxygen'].dropna().astype(str).tolist()
features_c = lasso_min_selected['CRP'].dropna().astype(str).tolist()
features_u = lasso_min_selected['Urea'].dropna().astype(str).tolist()

union_result = list(set(features_o) | set(features_c) | set(features_u))
union_result = [x for x in union_result if x != "" and x != "nan"]

exclude_vars = ["(Intercept)", "Aldehyd.Dehydrogenase.1.A1.pg.ml."]
ttt = [x for x in union_result if x not in exclude_vars]

ttt = [x.replace('.', '_') for x in ttt]
train.columns = train.columns.str.replace('.', '_')
test.columns = test.columns.str.replace('.', '_')


fixed_pred_base = ["age", "sex"]
fixed_pred_base = [x.replace('.', '_') for x in fixed_pred_base]
inter_pred = [x for x in ttt if x not in fixed_pred_base]

def make_formula(dep_var, fixed_vars, inter_vars, all_vars):
    fixed_part = " + ".join(fixed_vars)
    others = [v for v in all_vars if v not in fixed_vars]
    if others:
        fixed_part += " + " + " + ".join(others)
    
    inter_part = " + ".join([f"time:{x}" for x in inter_vars])
    return f"{dep_var} ~ {fixed_part} + time + {inter_part}"

form_o = make_formula("oxygen", fixed_pred_base, inter_pred, ttt)
form_c = make_formula("crp", fixed_pred_base, inter_pred, ttt)
form_u = make_formula("urea", fixed_pred_base, inter_pred, ttt)

print(f"Generated Formula (Oxygen): {form_o}")

Generated Formula (Oxygen): oxygen ~ age + sex + IGLV3_25 + IGHV3_60 + IGHV3_20 + IGLV3_10 + IGHV3_47 + IGLV3_9 + IGLV2_23 + IGHV3_43 + VEGF_pg_ml_ + CXCL10_IP_10_CRG_2_pg_ml_ + CCL22_MDC_pg_ml_ + CXCL4_PF4_pg_ml_ + IGKV1_27 + TRAJ60 + IGLV5_45 + IL_18_IL_1F4_pg_ml_ + CXCL9_MIG_pg_ml_ + IGLV3_19 + IGKV3_20 + IGLV4_69 + Granzyme_A_pg_ml_ + IGHV7_34_1 + IL_5_pg_ml_ + IGLJ5 + IGLV7_43 + IGLV2_11 + CCL23_MPIF_1_pg_ml_ + TRAV18 + TRBV13 + IGHV1_45 + CCL4_MIP_1_beta_pg_ml_ + TRAV11 + IL_33_pg_ml_ + IGHV3_35 + TRAJ57 + uPAR_pg_ml_ + IGLV4_60 + IL_6_pg_ml_ + IGHV1_2 + IGLV3_1 + TRBV6_9 + IGLV8_61 + TRBV5_1 + IGLV2_8 + CD14_pg_ml_ + MICA_pg_ml_ + IGLJ7 + IGHV1_68 + IGLV9_49 + IGLV6_57 + TRBV7_6 + TRBV23_1 + TRAV28 + MMP_7_pg_ml_ + IGFBP_1_pg_ml_ + CXCL1_GRO_alpha_pg_ml_ + IGLV1_47 + IL_2_pg_ml_ + TRBV7_1 + IGLV1_44 + IGHV1_46 + IGLV1_51 + TRBV6_4 + IL_3_pg_ml_ + IGHV3_64 + MMP_3_pg_ml_ + M_CSF_pg_ml_ + TRAJ58 + CCL19_MIP_3_beta_pg_ml_ + IGHV3_25 + BDNF_pg_ml_ + TRAJ33 + IGKV1_39 + IGLV10_54 + T

In [28]:
# -----------------------------------------------------------
# 3. Manual Linear Mixed Model Class (수정됨)
# -----------------------------------------------------------

class ManualLinearMixedModel:
    def __init__(self, formula, data, group_col):
        self.formula = formula
        self.data = data
        self.group_col = group_col
        
        self.y_df, self.X_df = patsy.dmatrices(formula, data, return_type='dataframe')
        self.y = self.y_df.iloc[:, 0].values.flatten()
        self.X = self.X_df.values
        self.groups = data[group_col].values
        self.unique_groups = np.unique(self.groups)
        
        self.p = self.X.shape[1]
        
        self.beta_hat = None
        self.sigma_b_hat = None
        self.sigma_e_hat = None
        self.residuals = None
        self.logLik = None
        self.n_params = None

    def neg_log_likelihood(self, params):
        beta = params[:self.p]
        sigma_b = np.exp(params[self.p])
        sigma_e = np.exp(params[self.p + 1])
        
        nll = 0
        for grp in self.unique_groups:
            mask = (self.groups == grp)
            y_i = self.y[mask]
            X_i = self.X[mask]
            n_i = len(y_i)
            
            resid = y_i - X_i @ beta
            
            # V_i = sigma_b^2 * J + sigma_e^2 * I
            V_i = (sigma_b**2) * np.ones((n_i, n_i)) + (sigma_e**2) * np.eye(n_i)
            
            try:
                sign, logdet = np.linalg.slogdet(V_i)
                inv_V_resid = np.linalg.solve(V_i, resid)
                quad_form = resid.T @ inv_V_resid
                nll += 0.5 * (logdet + quad_form + n_i * np.log(2 * np.pi))
            except np.linalg.LinAlgError:
                return np.inf
        return nll

    def fit(self):
        beta_init = np.linalg.lstsq(self.X, self.y, rcond=None)[0]
        initial_params = np.concatenate([beta_init, [0.0, 0.0]])
        result = minimize(self.neg_log_likelihood, initial_params, method='L-BFGS-B')
        
        self.opt_params = result.x
        self.beta_hat = self.opt_params[:self.p]
        self.sigma_b_hat = np.exp(self.opt_params[self.p])
        self.sigma_e_hat = np.exp(self.opt_params[self.p + 1])

        self.logLik = -result.fun
        self.n_params = len(result.x)
        
        self.fitted_values = (self.X @ self.beta_hat).flatten()
        self.residuals = (self.y - self.fitted_values).flatten() 
        
        return result

    def summary(self):
        print(f"Log-Likelihood: {-self.logLik:.2f}")
        print(f"Random Effect SD (Sigma_b): {self.sigma_b_hat:.4f}")
        print(f"Residual SD (Sigma_e): {self.sigma_e_hat:.4f}")

    def predict(self, newdata):
        _, X_new = patsy.dmatrices(self.formula, newdata, return_type='dataframe')
        return (X_new @ self.beta_hat).flatten()

In [6]:
# -----------------------------------------------------------
# 4. Manual Joint Model Class
# -----------------------------------------------------------

class ManualJointModel:
    def __init__(self, models_dict, data, group_col):
        self.models_dict = models_dict
        self.data = data
        self.group_col = group_col
        self.outcomes = list(models_dict.keys())
        self.num_outcomes = len(self.outcomes)
        self.y_list = []
        self.X_list = []
        self.beta_dims = [] 
        
        self.groups = data[group_col].values
        self.unique_groups = np.unique(self.groups)
        
        print(f"Initializing Joint Model for {self.num_outcomes} outcomes...")
        print(f"Target Structure: Correlation between {self.outcomes}")
        
        for name in self.outcomes:
            formula = models_dict[name]
            y_df, X_df = patsy.dmatrices(formula, data, return_type='dataframe')
            
            self.y_list.append(y_df.iloc[:, 0].values)
            self.X_list.append(X_df.values)
            self.beta_dims.append(X_df.shape[1])
            
        # (1) Fixed Effects (Beta)
        self.total_fixed_params = sum(self.beta_dims)
        
        # (2) Random Effects Covariance Matrix (D) - Unstructured
        self.num_cov_params = self.num_outcomes * (self.num_outcomes + 1) // 2
        
        # (3) Residual Variances (Sigma_e)
        self.num_resid_params = self.num_outcomes
        self.total_params = self.total_fixed_params + self.num_cov_params + self.num_resid_params

    def _unpack_params(self, params):
        """
        최적화 파라미터를 Beta, D(Covariance), R(Residuals)로 분해
        """
        curr = 0
        
        # 1. Fixed Effects (Betas)
        betas = []
        for dim in self.beta_dims:
            betas.append(params[curr : curr + dim])
            curr += dim
            
        # 2. Covariance Matrix D (Random Effects)
        L_params = params[curr : curr + self.num_cov_params]
        curr += self.num_cov_params
        L = np.zeros((self.num_outcomes, self.num_outcomes))
        indices = np.tril_indices(self.num_outcomes)
        L[indices] = L_params
        D = L @ L.T
        
        # 3. Residual Variances (log scale -> exp)
        log_resid = params[curr:]
        resid_sigmas = np.exp(log_resid)
        
        return betas, D, resid_sigmas

    def neg_log_likelihood(self, params):
        """
        Joint Negative Log-Likelihood 계산
        """
        betas, D, resid_sigmas = self._unpack_params(params)
        nll = 0
        
        for grp in self.unique_groups:
            mask = (self.groups == grp)
            n_i = np.sum(mask)
            
            # --- [1] Residual Vector (r_i) ---
            resid_parts = []
            for k in range(self.num_outcomes):
                y_k = self.y_list[k][mask]
                X_k = self.X_list[k][mask]
                resid_parts.append(y_k - X_k @ betas[k])
            
            r_i = np.concatenate(resid_parts)
            
            # --- [2] Design Matrix for Random Effects (Z_i) ---
            Z_i = np.zeros((n_i * self.num_outcomes, self.num_outcomes))
            for k in range(self.num_outcomes):
                row_start = k * n_i
                row_end = (k + 1) * n_i
                Z_i[row_start:row_end, k] = 1.0
            
            # --- [3] Construct Variance Matrix (V_i) ---
            # V_i = Z_i * D * Z_i.T + R_i
            R_blocks = [(resid_sigmas[k]**2) * np.eye(n_i) for k in range(self.num_outcomes)]
            R_i = block_diag(*R_blocks)
            V_i = Z_i @ D @ Z_i.T + R_i
            
            # --- [4] Log-Likelihood Calculation ---
            try:
                # Cholesky of V_i
                L_Vi = np.linalg.cholesky(V_i)
                logdet = 2 * np.sum(np.log(np.diag(L_Vi)))
                # y = V^-1 * r
                y_solve = np.linalg.solve(L_Vi, r_i)
                quad_form = np.sum(y_solve**2)
                nll += 0.5 * (logdet + quad_form)
                
            except np.linalg.LinAlgError:
                return np.inf
        
        total_obs = len(self.data) * self.num_outcomes
        nll += 0.5 * total_obs * np.log(2 * np.pi)
        
        return nll

    def fit(self):
        print("\n[Step 1] Estimating Initial Parameters (OLS)...")
        init_betas = []
        for k in range(self.num_outcomes):
            b_ols = np.linalg.lstsq(self.X_list[k], self.y_list[k], rcond=None)[0]
            init_betas.append(b_ols)
        
        x0_betas = np.concatenate(init_betas)
        L_init = np.eye(self.num_outcomes)
        indices = np.tril_indices(self.num_outcomes)
        x0_cov = L_init[indices]
        x0_resid = np.zeros(self.num_outcomes)
        x0 = np.concatenate([x0_betas, x0_cov, x0_resid])
        
        print("[Step 2] Starting Joint Optimization (L-BFGS-B)...")
        print(" -> Optimizing Correlations between Outcomes...")
        
        result = minimize(self.neg_log_likelihood, x0, method='L-BFGS-B',
                          options={'maxiter': 2000, 'disp': True})
        
        self.opt_params = result.x
        self.logLik = -result.fun
        self.aic = 2 * len(self.opt_params) - 2 * self.logLik
        self.final_betas, self.final_D, self.final_resid_sigmas = self._unpack_params(self.opt_params)
        
        print("Optimization Completed.")

    def summary(self):
        print("\n" + "="*60)
        print(" MANUAL JOINT MODEL RESULTS (Correlated Random Effects)")
        print("="*60)
        print(f"Log-Likelihood: {self.logLik:.4f}")
        print(f"AIC: {self.aic:.4f}")
        print("\n[1] Estimated Covariance Matrix (D):")

        D_df = pd.DataFrame(self.final_D, index=self.outcomes, columns=self.outcomes)
        print(D_df)
        print("\n[2] Estimated Correlation Matrix:")
        
        d_diag = np.sqrt(np.diag(self.final_D))
        corr_matrix = self.final_D / np.outer(d_diag, d_diag)
        corr_df = pd.DataFrame(corr_matrix, index=self.outcomes, columns=self.outcomes)
        print(corr_df)
        
        print("\n[3] Residual Standard Deviations (Sigma_e):")
        for k, name in enumerate(self.outcomes):
            print(f"  - {name}: {self.final_resid_sigmas[k]:.4f}")

In [30]:
# -----------------------------------------------------------
# 5. 실행 및 결과 확인 (Execution)
# -----------------------------------------------------------

# 5.1 개별 모델 테스트 (Oxygen)
print("\n[Test: Single Oxygen Model]")
model_o = ManualLinearMixedModel(form_o, train, group_col='ID')
model_o.fit()
model_o.summary()

print("\n[Test: Single CRP Model]")
model_c = ManualLinearMixedModel(form_c, train, group_col='ID')
model_c.fit()
model_c.summary()

print("\n[Test: Single Urea Model]")
model_u = ManualLinearMixedModel(form_u, train, group_col='ID')
model_u.fit()
model_u.summary()


[Test: Single Oxygen Model]
Log-Likelihood: 328.81
Random Effect SD (Sigma_b): 1.0000
Residual SD (Sigma_e): 1.0000

[Test: Single CRP Model]
Log-Likelihood: 371.59
Random Effect SD (Sigma_b): 1.0000
Residual SD (Sigma_e): 1.0000

[Test: Single Urea Model]
Log-Likelihood: 802.77
Random Effect SD (Sigma_b): 1.0000
Residual SD (Sigma_e): 1.0000


In [7]:
print("\n[Test: Joint Model with Covariance Structure]")

formulas = {
    'oxygen': form_o,
    'crp': form_c,
    'urea': form_u
}

joint_model = ManualJointModel(formulas, train, group_col='ID')
joint_model.fit()
joint_model.summary()


[Test: Joint Model with Covariance Structure]
Initializing Joint Model for 3 outcomes...
Target Structure: Correlation between ['oxygen', 'crp', 'urea']

[Step 1] Estimating Initial Parameters (OLS)...
[Step 2] Starting Joint Optimization (L-BFGS-B)...
 -> Optimizing Correlations between Outcomes...


/tmp/ipykernel_41471/2289036137.py:134: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  result = minimize(self.neg_log_likelihood, x0, method='L-BFGS-B',


Optimization Completed.

 MANUAL JOINT MODEL RESULTS (Correlated Random Effects)
Log-Likelihood: -1508.9884
AIC: 4151.9769

[1] Estimated Covariance Matrix (D):
              oxygen           crp          urea
oxygen  1.000000e+00 -5.101209e-12 -8.568044e-11
crp    -5.101209e-12  1.000000e+00 -3.400255e-11
urea   -8.568044e-11 -3.400255e-11  1.000000e+00

[2] Estimated Correlation Matrix:
              oxygen           crp          urea
oxygen  1.000000e+00 -5.101209e-12 -8.568044e-11
crp    -5.101209e-12  1.000000e+00 -3.400255e-11
urea   -8.568044e-11 -3.400255e-11  1.000000e+00

[3] Residual Standard Deviations (Sigma_e):
  - oxygen: 1.0000
  - crp: 1.0000
  - urea: 1.0000
